In [1]:
import pandas as pd

In [2]:
# A chardet modul importálása, amely segít a fájl karakterkódolásának detektálásában
import chardet

# Megnyitjuk az "IMDB_Dataset.csv" fájlt bináris ("rb") módban
with open("IMDB_Dataset.csv", "rb") as f:
    # Beolvassuk a teljes fájl tartalmát nyers (raw) formában
    rawdata = f.read()

    # A chardet detect() függvényével megpróbáljuk meghatározni a fájl karakterkódolását
    result = chardet.detect(rawdata)

# Kiírjuk a detektált karakterkódolást a konzolra
print(result["encoding"])


utf-8


In [3]:
import re

def process_csv(file_path):
    data = []

    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()  # Sorvégi whitespace-ek eltávolítása

            # Keresünk egy "positive" vagy "negative" szót
            match = re.search(r"\b(positive|negative)\b", line, re.IGNORECASE)

            if match:
                before_text = line[:match.start()].strip()  # Minden, ami előtte van
                sentiment = match.group(1).lower()  # "positive" vagy "negative"
                sentiment = 1 if sentiment == "positive" else 0  # Cseréljük le számokra
                data.append([before_text, sentiment])
            else:
                data.append([line, None])  # Ha nincs "positive"/"negative", második oszlop üres

    # Adatok átalakítása DataFrame formátumba
    df = pd.DataFrame(data, columns=["SentimentText", "Sentiment"])
    return df

In [4]:
df = process_csv("IMDB_Dataset.csv")
print(df.head())

                                       SentimentText  Sentiment
0                                   review,sentiment        NaN
1  "One of the other reviewers has mentioned that...        1.0
2  "A wonderful little production. <br /><br />Th...        1.0
3  "I thought this was a wonderful way to spend t...        1.0
4  "Basically there's a family where a little boy...        0.0


In [5]:
train_positive=df[df["Sentiment"]==1]
train_negative=df[df["Sentiment"]==0]

In [6]:
train_positive.to_csv("train_positive.csv", index=False)
train_negative.to_csv("train_negative.csv", index=False)

In [7]:
# Az re modul importálása, amely reguláris kifejezések (regex) kezelésére szolgál
import re

# Az nltk könyvtárból importáljuk a stopwords modult, amely előre definiált gyakori szavakat tartalmaz különböző nyelveken
from nltk.corpus import stopwords

# Az nltk könyvtárból importáljuk a word_tokenize függvényt, amely szavakra bontja a szöveget
from nltk.tokenize import word_tokenize

In [8]:
# Az nltk könyvtár importálása, amely természetes nyelvfeldolgozási (NLP) feladatokhoz használható
import nltk

# A 'stopwords' adatcsomag letöltése, amely tartalmazza a különböző nyelvek gyakori, jelentéssel nem bíró szavait
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
stop_words = set(stopwords.words('english'))

In [10]:
regex_pattern = re.compile(
    r"""
    (?:@[\w_]+)                   # Remove Twitter usernames (@username)
    | (?:\#+[\w_]+[\w\'_\-]*)     # Remove Twitter hashtags (#hashtag)
    | <[^>]+>                     # Remove HTML tags
    | [<>]?[:;=8][\-o\*\']?[\)\]\(\[dDpP/\:\}\{@\|\\]  # Remove emoticons
    | [\)\]\(\[dDpP/\:\}\{@\|\\][\-o\*\']?[:;=8][<>]?  # Remove emoticons (reverse)
    | [^\w\s]                     # Remove special characters (except letters and spaces)
    """,
    re.VERBOSE | re.IGNORECASE # VERBOSE: könnyebb olvashatóság, IGNORECASE: kis- és nagybetűk figyelmen kívül hagyása
)

In [11]:
def tokenize_text(text):
    # Az egész szöveget kisbetűssé alakítjuk
    text = text.lower()

    # Speciális karakterek eltávolítása a korábban definiált reguláris kifejezés segítségével
    text = regex_pattern.sub('', text)

    # A szöveget szavakra bontjuk (tokenizáljuk)
    tokens = word_tokenize(text)

    # Stop szavak eltávolítása (pl. „és”, „vagy”, „a” stb.)
    tokens = [word for word in tokens if word not in stop_words]

    # A megtisztított és tokenizált szavak listáját visszaadjuk
    return tokens

In [12]:
def tokenize_dataframe(df):
    # Létrehozunk egy üres listát az összes token számára
    all_tokens = []

    # Végigmegyünk az adathalmaz 'SentimentText' oszlopán (szövegeken)
    for sentence in df['SentimentText']:
        # Tokenizáljuk az aktuális mondatot a tokenize_text() függvény segítségével
        tokens = tokenize_text(sentence)

        # Az eredményül kapott tokeneket hozzáadjuk az összes token listájához
        all_tokens.extend(tokens)

    # Visszaadjuk az összegyűjtött tokenek listáját
    return all_tokens

In [13]:
import nltk
nltk.download('punkt_tab') # A 'punkt' modul letöltése, amely szükséges a mondatok és szavak tokenizálásához

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [14]:
tokenized_pos=tokenize_dataframe(train_positive)
tokenized_neg=tokenize_dataframe(train_negative)

In [15]:
# Az sklearn könyvtárból importáljuk a CountVectorizer osztályt,
# amely segít a szövegbeli szavak előfordulásának megszámlálásában
from sklearn.feature_extraction.text import CountVectorizer

# Függvény, amely egy tokenizált szólistából létrehoz egy lexikont (a leggyakoribb szavak listáját)
def build_lexicon_from_tokens(tokenized_words, top_n=2000):

    # A tokenizált szavakat egyetlen nagy sztringgé alakítjuk szóközökkel elválasztva
    text = " ".join(tokenized_words)

    # Létrehozunk egy CountVectorizer objektumot, amely segít a szavak gyakoriságának számolásában
    cv = CountVectorizer()

    # A szöveget átalakítjuk egy számláló mátrixszá (Bag-of-Words modell)
    X = cv.fit_transform([text])

    # Az összes szót megszámoljuk és listába konvertáljuk
    word_counts = X.sum(axis=0).tolist()[0]

    # A szavakat összepárosítjuk az előfordulási számukkal
    words_with_counts = list(zip(cv.get_feature_names_out(), word_counts))

    # A szavakat gyakoriság szerint csökkenő sorrendbe rendezzük
    sorted_words = sorted(words_with_counts, key=lambda x: x[1], reverse=True)

    # Kiválasztjuk a top_n leggyakoribb szót
    top_words = sorted_words[:top_n]

    # A top_words listát szavak és előfordulási számok szerint két külön listára bontjuk,
    # de csak a szavakat tartjuk meg
    top_words, _ = zip(*top_words)

    # Visszaadjuk a top_n leggyakoribb szót tartalmazó listát
    return top_words

In [16]:
tokenized_pos=build_lexicon_from_tokens(tokenized_pos)
tokenized_neg=build_lexicon_from_tokens(tokenized_neg)

In [17]:
unique_pos = set(tokenized_pos).difference(set(tokenized_neg))
unique_neg = set(tokenized_neg).difference(set(tokenized_pos))

In [18]:
lexicon = dict.fromkeys(unique_pos, 1)
lexicon.update(dict.fromkeys(unique_neg, 0))

In [19]:
lexicon

{'studios': 1,
 '710': 1,
 'traditional': 1,
 'smile': 1,
 'cartoons': 1,
 'strongly': 1,
 '90s': 1,
 'touching': 1,
 'stunning': 1,
 'tragedy': 1,
 'keeping': 1,
 'chosen': 1,
 'bringing': 1,
 'fame': 1,
 'civil': 1,
 'influence': 1,
 'subtle': 1,
 'vhs': 1,
 'crowd': 1,
 'gritty': 1,
 'humanity': 1,
 'anne': 1,
 'decade': 1,
 'von': 1,
 'friendship': 1,
 'kong': 1,
 'recognize': 1,
 'visuals': 1,
 'freedom': 1,
 'victor': 1,
 'noir': 1,
 'fox': 1,
 '50s': 1,
 'reach': 1,
 'versions': 1,
 'dealing': 1,
 'greater': 1,
 'meanwhile': 1,
 'colors': 1,
 'funniest': 1,
 'spectacular': 1,
 'surreal': 1,
 'strength': 1,
 'suspenseful': 1,
 'debut': 1,
 'deeply': 1,
 'remembered': 1,
 'originally': 1,
 'realism': 1,
 'tragic': 1,
 'appreciated': 1,
 'viewed': 1,
 'emotionally': 1,
 'gripping': 1,
 'marry': 1,
 'foreign': 1,
 'reaction': 1,
 'germany': 1,
 'effectively': 1,
 'dennis': 1,
 'built': 1,
 'universe': 1,
 'mainstream': 1,
 'alex': 1,
 'generation': 1,
 'satisfying': 1,
 'grand': 1,


In [20]:
print(type(lexicon))

<class 'dict'>


In [21]:
import json

with open("data.txt", "w") as file:
    json.dump(lexicon, file, indent=4)

In [22]:
import chardet

with open("train.csv", "rb") as f:
    rawdata = f.read()
    result = chardet.detect(rawdata)

print(result["encoding"])

Windows-1252


In [23]:
test_data=pd.read_csv('train.csv', sep=',', encoding='Windows-1252')

In [24]:
test_data

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...
...,...,...,...
99984,99996,0,@Cupcake seems like a repeating problem hop...
99985,99997,1,@cupcake__ arrrr we both replied to each other...
99986,99998,0,@CuPcAkE_2120 ya i thought so
99987,99999,1,@Cupcake_Dollie Yes. Yes. I'm glad you had mor...


In [25]:
test_data = test_data.drop(columns=["ItemID"])

In [27]:
# Az sklearn könyvtárból importáljuk a CountVectorizer osztályt, amely a szöveget számszerűsített jellemzőkké alakítja (Bag-of-Words modell)
from sklearn.feature_extraction.text import CountVectorizer

# Az sklearn könyvtárból importáljuk a MultinomialNB osztályt, amely egy Naiv Bayes osztályozó kifejezetten szövegalapú osztályozásra
from sklearn.naive_bayes import MultinomialNB

# Az sklearn könyvtárból importáljuk az accuracy_score függvényt, amely az osztályozó pontosságát méri
from sklearn.metrics import accuracy_score

# Az sklearn könyvtárból importáljuk a make_pipeline függvényt, amely segít egyszerűsített módon létrehozni egy gépi tanulási folyamatot
from sklearn.pipeline import make_pipeline

# Importáljuk a train_test_split függvényt az sklearn.model_selection modulból
from sklearn.model_selection import train_test_split

# Az adathalmazból kiválasztjuk az osztályozandó szövegeket (X) és a hozzájuk tartozó címkéket (y)
X = test_data["SentimentText"]  # Szöveges adatok
y = test_data["Sentiment"]  # Címkék (pl. pozitív vagy negatív vélemények)

# Az adathalmazt két részre osztjuk: tanító (train) és teszt (test) adathalmazra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# - A `test_size=0.2` azt jelenti, hogy az adatok 20%-át a tesztelésre tartjuk fenn, 80%-át pedig a modell betanítására használjuk.
# - A `random_state=42` biztosítja, hogy az adatok mindig ugyanúgy legyenek felosztva, így az eredmények reprodukálhatók.

# Létrehozunk egy CountVectorizer objektumot, amely a szövegeket számszerűsített vektorokká alakítja
vectorizer = CountVectorizer()

# Létrehozunk egy Naiv Bayes osztályozót, amely alkalmas szövegek kategorizálására
model = MultinomialNB()

# Egyéni tokenizáló függvény létrehozása
def custom_tokenizer(text):
    # A szöveget kisbetűssé alakítjuk, majd szavakra bontjuk szóközök mentén
    tokens = text.lower().split()

    # Csak azokat a szavakat tartjuk meg, amelyek szerepelnek a korábban létrehozott lexikonban
    filtered_tokens = [token for token in tokens if token in lexicon]

    # A megtisztított szavak listáját visszaadjuk
    return filtered_tokens

In [28]:
# Pipeline létrehozása, amely először vektorizál, majd betanítja a modellt
pipeline = make_pipeline(CountVectorizer(tokenizer=custom_tokenizer), model)

In [29]:
# A modell betanítása a train adatokon
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('countvectorizer',
                 CountVectorizer(tokenizer=<function custom_tokenizer at 0x792221b5e480>)),
                ('multinomialnb', MultinomialNB())])

In [30]:
# A modell tesztelése
y_pred = pipeline.predict(X_test)

In [31]:
# A tesztadatok kiértékelése
accuracy = accuracy_score(y_test, y_pred)

In [32]:
accuracy

0.5763576357635763